# Tutorial notebook for GRIP-Tomo
**August George, PNNL, 2022**

A quick tutorial on how to convert a pdb into graph, convert a density (.mrc) into a graph, and classify graphs

For more in-depth information on each module, subroutine, and their inputs/outputs see the API reference in `/docs`

All the data used here can be found in the `example_data` folder which contains a readme describing the files in more detail.

### Importing modules

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os, sys
from pathlib import Path
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd

sys.path.insert(1, os.path.join(sys.path[0], '..'))  # add parent directory to path (/scripts) to import modules
plt.rcParams['figure.figsize'] = [15, 10]  # make plots larger
data_path = Path(Path(os.getcwd()).parent,'example_data')  # datafile path

# import grip-tomo modules
import pdb2graph as p2g
import density2graph as d2g
import graph2class as g2c

print("modules imported  \u2713")

# Using `pdb2graph.py`:  pdb --> graph

Converting an alpha helix bundle protein `3vjf.pdb` (found in `example_data`) into a graph, using an 8 Angstrom pairwise distance threshold for alpha carbons

#### step 1: convert pdb to pandas dataframe using the `PDB_to_df()` subroutine 

Each alpha carbon (or atom) x,y,z coordinate is stored as well as residue ID and hydrophobicity

`PDB_to_df()` requires the following inputs:
- `pdb_code` is the name of protein (for output)
- `fname` is the pdb file to turn into graph
- `pdbx` is a flag to set if using .pdb (0) or .pdbx (1) file format
- `o` is the residue indexing offest (default = 0). this can typically be kept = 0
- `CA_only` is a flag to set if using only alpha carbons (1) or all atoms (0)

In [ ]:
### set input parameters
pdb_code = '3vjf' # name of protein (for output)
fname = Path(data_path,'3vjf.pdb')  # file to turn into graph
pdbx = 0  # using .pdb (0) or .pdbx (1) file format
o = 0  # residue indexing offest (default = 0)
CA_only = 1  # only use alpha carbons for graph nodes

### convert from pdb file into dataframe
df = p2g.PDB_to_df(pdb_code, fname, pdbx, o)  # convert pdb file into dataframe of alpha carbon coordinates and s/n 

### display results
print('\ndataframe from PDB\n')
print(df)

#### step 2: convert pandas dataframe into a networkx graph using the `PDB_df_to_G()` subroutine 

Graph nodes are assigned for each atom / alpha carbon x,y,z coordinate. The pairwise Euclidean distance between each atom / alpha carbon is calculated and pairs with a distance below a threshold are assigned edges. 

`PDB_df_to_G()` requires the following inputs:
- `df` is the dataframe containing the x,y,z coordinates of the nodes
- `d_cut` is the maximum Euclidean distance allowed between nodes to assign edges, in Angstroms

use `d_cut` = 8 Angstroms as the distance cutoff.


In [ ]:
### set input parameter
d_cut = 8 # pairwise distance cutoff for assigning edges, in Angstroms

### convert from dataframe to graph
G = p2g.PDB_df_to_G(df, d_cut)  # convert coordinate dataframe into network graph

### display results
plt.title('3vjf: pdb to graph')
nx.draw(G, pos=nx.spring_layout(G), node_size=20)

#### step 3 [optional]: save the data as a graph xml file (.gexf) and datafile (.csv) using the `save_data()` subroutine 

`save_data()` requires the following inputs:
- `df` is the dataframe containing the x,y,z coordinates of the nodes
- `G` is the networkx graph
- `df_outname` is the output filename for the dataframe
- `G_outname` is the outputfilename for the graph

_note: the data is saved in your current directory. to save in a specific directory use the `save_data_at_this_folder()` function which requires a filepath as the first input argument._ 


In [ ]:
# save files (optional)
# p2g.save_data(df, G, '3vjf_data', '3vjf_graph')  # save dataframe as .csv and graph as .gexf (current directory)
# p2g.save_data_at_this_folder('your/path/goes/here', df, G, '3vjf_data', '3vjf_graph')  # save dataframe as .csv and graph as .gexf 

# Using `density2graph.py`: 3D density .mrc-->graph

Below is a an example using the 3VJF density file `3vjf_density.mrc`. 

#### step 1: load mrc file and normalize data using the `load_density_file()` and `normalize_and_threshold_data()` subroutines 

The density values are normalized and a threshold cutoff is applied to remove pixels with a density value below the threshold

`load_density_file()` requires a `filename` as an input argument. 

`normalize_and_threshold_data()` requires the following input arguments:
- `mrc` is  mrcfile data object (using the mrcfile python package)
- `t` is the pixel intesity cutoff threshold
- `noise_stdev` is the standard deviation of white Gaussian noise to add to the data. Default = 0 (no noise added)
- `norm_T` is a Boolean flag that sets if the inputted threshold is a (raw) unnormalized value, or a normalized value (0,1). Default is False.

In [ ]:
## set input parameters
fname =  Path(data_path,'3vjf_density.mrc')
t = 0.5  # pixel intensity threshold - unnormalized

### load density file, normalize the data and threshold, and extract the x,y,z coordinates of the remaining pixels
mrc = d2g.load_density_file(fname)  # load density file
xyz_data = d2g.normalize_and_threshold_data(mrc,t)  # normalize data and threshold and then apply threshold

### display results
print('x,y,z coordinates for high intensity pixels in density (.mrc)\n')
print(xyz_data)

#### step 2: cluster data and get cluster centroids using the `cluster_data()` and `get_cluster_centroids()` subroutines 

The x,y,z data of the remaining density after thresholding is clustered using the DBSCAN algorithm in scikit-learn: [ref](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html)

Afterwords, the centroid coordinates of these clusters is calculated to be used as the graph nodes.

`cluster_data()` requires the following input arguments:
- `xyz_data` is an array of xyz coordinates from the density in the form of: A[0] = [x0,y0,z0]
- `DBSCAN_epsilon` is the maximum distance between two samples for one to be considered as in the neighborhood of the other
- `DBSCAN_min_samples` is the number of samples (or total weight) in a neighborhood for a point to be considered as a core point

`get_cluster centroids()` requires the following input arguements:
- `xyz_data` is an array of xyz coordinates from the density in the form of: A[0] = [x0,y0,z0]
- `model` is a sklearn DBSCAN clustering object 

`DBSCAN_epsilon` = 1, and `DBSCAN_min_samples`= 4 where found empirically to give good results

In [ ]:
### set clustering parameters
DBSCAN_epsilon = 1  # DBSCAN  epsilon
DBSCAN_min_samples = 4  # DBSCAN min samples

### perform clustering and get cluster centers
model = d2g.cluster_data(xyz_data,DBSCAN_epsilon,DBSCAN_min_samples)  # cluster thresholded data using DBSCAN
coarse_model = d2g.get_cluster_centroids(xyz_data,model)  # coarse grain model by getting cluster centroids

### Plot the results
print('clustering results:')
fig = d2g.plot_clustering_results(xyz_data, coarse_model, figsize=10)  # plot results of clustering and coarse-graining

#### step 3: create graph from cluster centroids using `create_and_save_graph()` subroutine

Using each cluster centroid as a graph node, we assign edges between nodes if their pairwise Euclidean distance (in pixels) is not above a cutoff threshold.

`create_and_save_graph()` requires the following input arguments:
- `coarse_model` is an array of cluster centroids where A[0] = [centroid_x0, centroid_y0, centroid_z0]
- `d_cut` is the pairwise distance cutoff for assigning edges to nodes, in pixels.
- `out_fname` is the filename for output
- `save` is a flag to save file (True) or not (False)

Assuming we want an 8 Angstrom cutoff distance, and 1 voxel = 1 Angstrom, we use a cutoff distance of 8 voxels (d_cut=8). 

In [ ]:
### set input parameters
d_cut = 8  # pairwise cutoff distance in voxels (edge is assigned to two nodes if Euclidean distance <= cutoff distance)

### generate graph (use save=True to save a .gexf file)
G = d2g.create_and_save_graph(coarse_model, d_cut, 'density2graph', save=False)  # create graph where nodes = centroids, edges assigned by pairwise cutoff

### output results
plt.title('3vjf density to graph:')
nx.draw(G, pos=nx.spring_layout(G), node_size=20)

# Using `graph2class.py`: graph --> class



Comparing the similarity of graph network features from different protein classes

The 'control' protein graph `3vjf_pdb2graph.gexf` was generated directly from the pdb structure `3vjf.pdb` using `pdb2graph.py`. 

The 'sample' protein graph `3vjf_density.gexf` was generated from a 3D volume density of 3VFJ and converted into a graph using `density2graph.py`.

This approach can be extended to multiple 'control' protein graphs and 'sample' protein graphs

For each graph, several network features are calculated:
- 'n nodes',
- 'n edges',
- 'density',
- 'diameter',
- 'avg path length',
- 'avg clustering',
- 'max closeness centrality',
- 'max eigenvector centrality',
- 'max betweenness centrality',
- 'degree assortativity',
- 'max clique number',
- 'n communities',

#### step 1: set up which graph files and graph features to use for similarity checks


In [ ]:
### set input parameters
class_files = ['3vjf_pdb2graph.gexf']
sample_files = [
        '3vjf_density2graph.gexf', 
        ]
similarity_features_list = [
            'n nodes',
            'n edges',
            'density',
            'diameter',
            'avg path length',
            'avg clustering',
            'max closeness centrality',
            'max eigenvector centrality',
            'max betweenness centrality',
            'degree assortativity',
            'max clique number',
            'n communities',
    ]

### get file list for graph 'classes' and 'samples'
class_list = [Path(data_path, fname) for fname in class_files]
sample_list = [Path(data_path, fname) for fname in sample_files]

### output results
print('class graph files:\n')
print(class_list)
print('\nsample graph files:\n')
print(sample_list)

### step 2: calculating graph similarity using the `classify_graphs()` subroutine

For each class / sample graph pair, calculate (1 - relative distance) between each graph feature and then return the average across all features. 

`classify_graphs()` requires the following input arguments:
- `class_list`  is a list of control/reference graph files (classes)
- `sample_list` is a list of non-control/non-reference graph files (samples)
- `feature_list` is a list of which features to use for similarity score and must be a valid key to the graph features dictionary/dataframe

_note: you can change which subset of features to include in the scoring function by adjusting the `similarity_features_list` parameter._

In [ ]:
### classify based on network feature similarities
classify_df = g2c.classify_graphs(class_list, sample_list, similarity_features_list)

### display results
print('similarity scores:')
print(classify_df)

# next steps: review API documentation in `/docs` 
- plot all atom vs alpha carbon only using `plot_FA_and_CA_coordinates()` in `pdb2graph.py`
- add Gaussian white noise to density data using `add_Gaussian_noise()` in `density2graph.py`
- get `y_true` and `y_pred` for [scikit-learn classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html#sklearn.metrics.classification_report) using `process_similarity_df()` in `graph2class.py`